# Swin transformer

Ce notebook analyse le papier [Swin Transformer: Hierarchical Vision Transformer using Shifted Windows](https://arxiv.org/pdf/2103.14030) proposant une amélioration de l'archicture transformer avec un design hierarchique spécifique aux images pouvant rappeler les réseaux de neurones convolutifs.   
La première partie du notebook explique les propositions de l'article une par une et la seconde partie est une implémentation simplifiée de l'architecture.

<img src="images/swin.png" alt="swin" width="700"/> 


## Analyse de l'article 

L'idée principale du papier est d'appliquer l'attention de manière hierarchique sur des parties de plus en plus grande de l'image. Cette approche a plusieurs fondements : Tout d'abord, l'analyse des images en regardant d'abord les détails locaux avant de regarder la relation entre tous les pixels de l'image est intuitivement logique (c'est pourquoi les CNNs sont si performants). Ensuite, le fait que les tokens (patch) ne communique pas avec tous les autres permet d'améliorer le temps de calcul.  

### Architecture hierarchique

L'architecture hierarchique du swin transformer est résumée dans cette figure :  

<img src="images/hierarchique.png" alt="hierarchique" width="500"/> 

Nous avons vu, dans notre implémentation, que le modèle ViT convertit les patchs en "tokens" et applique simplement un transformer encoder sur tous les éléments. C'est une architecture très simple et sans aucun biais sur les données (c'est l'architecture de base du transformer qui peut s'appliquer sur un peu tous les types de données).

L'architecture swin ajoute un biais destinée à la rendre plus performante sur les images et plus rapide en terme de traitement. Comme on le voit dans la figure, l'image est d'abord séparé en très petits patchs (taille $4 \times 4$ dans le papier) et les patchs sont regroupés en fenêtre. La couche d'attention est ensuite appliquée uniquement sur chaque fenêtre de manière indépendante. Plus on va profond dans le réseau, plus la dimension C, taille des patchs (relative à l'image) et des fenêtres augmente jusqu'à avoir une fenêtre de toute l'image et le même nombre de patchs que l'architecture ViT. A la manière d'un CNN, le réseau traîte d'abord les informations locales puis au fur et à mesure (avec l'augmentation du receptive field) des informations de plus en plus globales. Cela se fait en augmentant le nombre de filtres et en diminuant la résolution de l'image. 

Les nouveaux blocks de transformer correspondants sont appelés "Window Multi-Head Self-Attention" (W-MSA dans le papier, attention le M signifie Multi-Head et pas Masked).


### Fenêtre glissante 

Dans leur analogie avec le CNN, les auteurs se sont rendu compte qu'il peut être problématique de séparer en fenêtres à des positions arbitraires car cela brise la connexion entre des pixels voisins situés à des extremités de fenêtres.   

Pour corriger ce problème, les auteurs proposent d'utiliser un système de fenêtre glissante (shifting window) dans chaque block swin. Les blocks swin sont agencés par paires comme décrit dans la figure du début du notebook.  

Voici à quoi ressemble la fenêtre glissante : 

<img src="images/shifting.png" alt="shifting" width="500"/>

Comme vous pouvez le constater, avec cette technique, on passe de $2 \times 2$ patchs à $3 \times 3$ patchs (de manière générale de $n \times n$ patchs à $(n+1) \times (n+1)$) ce qui est problématique pour le traitement par le réseau en particulier en batch.  

Les auteurs proposent d'incorporer un cyclic shift qui consiste à faire cette opération sur l'image pour permettre un traitement plus efficace : 

<img src="images/cyclic.png" alt="cyclic" width="500"/>

A noter que pour utiliser cette méthode, il est nécessaire de masquer les informations de patchs ne provenant pas d'une même partie de l'image. Les parties blanches, jaunes, vertes et bleues de la figure ne communique pas ensemble grâce à une couche d'attention masquée. 


### Relative position bias

L'architecture ViT utilisait un position embedding absolu pour ajouter une information de position sur les différents patchs. Le problème de position embedding est qu'il ne capture par les relations entre les patchs et est donc moins performant si on donne au transformer des images de résolutions différentes.

Le swin tranformer utilise un biais de positions relative pour compenser cela. Ce biais va dépendre de la distance relative entre les différents patchs. Ce biais est ajouté lorsque l'attention est calculée entre deux patchs. Ce biais a plusieurs effets mais son principal intêret est qu'il améliorer la capture des relations spatiales et qu'il permet de s'adapter à des images de résolutions différentes.

### Détails supplémentaires sur l'architecture

- Comme on le voit sur la première figure du notebook, il y a plus de couches dans le stage 3 du swin transformer. Lorsqu'on augmente le nombre de couches du réseaux, c'est uniquement les couches du stage 3 qui vont être augmentée, les autres couches restent fixes. Cela permet de béneficier de l'archicture swin (shifting etc ...) tout en étant suffisament profond et performant en terme de temps de traitement.

- Supposons que chaque fenêtre contienne des patchs de $M \times M$. La complexité computationnelle d'une couche multi-head self-attention (MSA) et celle d'une couche window multi-head self-attention (W-MSA) une image de $h \times w$ patchs sont:   
$\Omega(\text{MSA}) = 4hwC^2 + 2(h w)^2 C$    
$\Omega(\text{W-MSA}) = 4hwC^2 + 2M^2hwC$   
Le premier est de complexité quadratique tandis que le second est linéaire si $M$ est fixe. L'architecture swin permet de gagner en vitesse de traitement.